<a href="https://colab.research.google.com/github/lewisnjue/kaggle_competition_house_price/blob/main/house_price_prediction_pytorch_competion_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# let me upload my kaggle api key programatically
from google.colab import files
files.upload()
# this is the programmatically way to putting your api keys to google colab use it in your code

{}

In [ ]:
!pip install kaggle
''' for many casses teh kaggle is aready installed  '''
# this will install kaggle

' for many casses teh kaggle is aready installed  '

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
# i have also changed the permission of the file for me to read and write but others have no permission to it

In [ ]:
# download the zip file now
!kaggle competitions download -c house-prices-advanced-regression-techniques


house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# unzip the file like this
!unzip house-prices-advanced-regression-techniques.zip


Archive:  house-prices-advanced-regression-techniques.zip
replace data_description.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [74]:
!ls

data_description.txt				 sample_data		test.csv
house-prices-advanced-regression-techniques.zip  sample_submission.csv	train.csv


In [125]:
# create a dataframe for my train data import the libraries am going to use
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
%matplotlib inline
import torch
from torch import nn

import pandas as pd
df = pd.read_csv('train.csv',index_col='Id')


In [126]:
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [127]:
len(list(df.columns))

80

In [128]:
(df.isna()).sum()

,0
MSSubClass,0
MSZoning,0
LotFrontage,259
LotArea,0
Street,0
...,...
MoSold,0
YrSold,0
SaleType,0
SaleCondition,0


In [129]:
df.describe()


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [130]:
''' numerical_cols = df.select_dtypes(include=np.number).columns # this will get me the numerical columns
for col in numerical_cols:
    if df[col].isnull().any():
        df[col] = df[col].fillna(df[col].mean())

# Categorical columns with null values dropped
categorical_cols = df.select_dtypes(exclude=np.number).columns
for col in categorical_cols:
    if df[col].isnull().any():
        df.dropna(subset=[col], inplace=True) '''

numerical_columns = df.select_dtypes(include=np.number).columns
categorical_columns = df.select_dtypes(exclude=np.number).columns
list(numerical_columns),list(categorical_columns)
df[list(numerical_columns)] = df[list(numerical_columns)].fillna(df[list(numerical_columns)].mean())
df[list(categorical_columns)] = df[list(categorical_columns)].fillna('Missing')
df.head()


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,Missing,Reg,Lvl,AllPub,Inside,...,0,Missing,Missing,Missing,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,Missing,Reg,Lvl,AllPub,FR2,...,0,Missing,Missing,Missing,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,Missing,IR1,Lvl,AllPub,Inside,...,0,Missing,Missing,Missing,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,Missing,IR1,Lvl,AllPub,Corner,...,0,Missing,Missing,Missing,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,Missing,IR1,Lvl,AllPub,FR2,...,0,Missing,Missing,Missing,0,12,2008,WD,Normal,250000


In [131]:
df.select_dtypes(include=np.number).columns

Index(['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond',
       'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2',
       'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
       'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF',
       'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal',
       'MoSold', 'YrSold', 'SalePrice'],
      dtype='object')

In [132]:
df.select_dtypes(exclude=np.number).columns

Index(['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual',
       'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature',
       'SaleType', 'SaleCondition'],
      dtype='object')

In [133]:
df['SalePrice'].isna().sum()

0

In [134]:
# now in my data i have no null values -> this is very good now for my data predictions
''' let me now separate the target columns form the train columns '''
target = df['SalePrice']
labels = df.drop('SalePrice',axis=1)

In [135]:
target.shape,labels.shape

((1460,), (1460, 79))

In [136]:
''' let me normilize my data '''
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
numerical_cols_no_target = list(numerical_columns)[:-1]
labels[numerical_cols_no_target] = scaler.fit_transform(labels[numerical_cols_no_target])
labels.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1,0.073375,RL,-0.229372,-0.207142,Pave,Missing,Reg,Lvl,AllPub,Inside,...,-0.270208,-0.068692,Missing,Missing,Missing,-0.087688,-1.599111,0.138777,WD,Normal
2,-0.872563,RL,0.451936,-0.091886,Pave,Missing,Reg,Lvl,AllPub,FR2,...,-0.270208,-0.068692,Missing,Missing,Missing,-0.087688,-0.489110,-0.614439,WD,Normal
3,0.073375,RL,-0.093110,0.073480,Pave,Missing,IR1,Lvl,AllPub,Inside,...,-0.270208,-0.068692,Missing,Missing,Missing,-0.087688,0.990891,0.138777,WD,Normal
4,0.309859,RL,-0.456474,-0.096897,Pave,Missing,IR1,Lvl,AllPub,Corner,...,-0.270208,-0.068692,Missing,Missing,Missing,-0.087688,-1.599111,-1.367655,WD,Abnorml
5,0.073375,RL,0.633618,0.375148,Pave,Missing,IR1,Lvl,AllPub,FR2,...,-0.270208,-0.068692,Missing,Missing,Missing,-0.087688,2.100892,0.138777,WD,Normal


In [138]:
'''  do one hot encoding  for our categorical columns '''
labels = pd.get_dummies(labels,drop_first=True,dtype=int)

labels.select_dtypes(exclude=np.number).columns # that is the output i wanted , all columns should have numerical data tupes
labels.shape

(1460, 260)

In [139]:
'''  now convert my labels to tensor excluding the id column using pytorch '''
labels = torch.from_numpy(labels.values).float()
target = torch.from_numpy(target.values).float()

In [140]:
labels

tensor([[ 0.0734, -0.2294, -0.2071,  ...,  0.0000,  1.0000,  0.0000],
        [-0.8726,  0.4519, -0.0919,  ...,  0.0000,  1.0000,  0.0000],
        [ 0.0734, -0.0931,  0.0735,  ...,  0.0000,  1.0000,  0.0000],
        ...,
        [ 0.3099, -0.1840, -0.1478,  ...,  0.0000,  1.0000,  0.0000],
        [-0.8726, -0.0931, -0.0802,  ...,  0.0000,  1.0000,  0.0000],
        [-0.8726,  0.2248, -0.0581,  ...,  0.0000,  1.0000,  0.0000]])

In [141]:
labels.shape

torch.Size([1460, 260])

In [142]:
labels.dtype

torch.float32

In [150]:
target

tensor([208500., 181500., 223500.,  ..., 266500., 142125., 147500.])

In [145]:
''' create a model  '''
class regression_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features=260, out_features=200),  # Adjusted from 331 to 260
            nn.Linear(in_features=200, out_features=100),
            nn.Linear(in_features=100, out_features=50),
            nn.Linear(in_features=50, out_features=1))

    def forward(self, x):
        return self.layers(x)

In [146]:
''' create a loss function and an optimzer '''
model = regression_model()
loss_fun = nn.MSELoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.1)

In [149]:
''' do some dummy prediction '''
y_pred = model(labels.float())
y_pred.squeeze()

tensor([-0.0674, -0.0191, -0.0872,  ..., -0.0422, -0.1351, -0.0325],
       grad_fn=<SqueezeBackward0>)

In [151]:
# let now train our model
torch.manual_seed(42)
epochs = 1000
for epoch in range(epochs):
    model.train()# put the model in the training mode
    y_pred = (model(labels.float())).squeeze()
    loss = loss_fun(y_pred,target.float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    model.eval()
    with torch.inference_mode():
        y_pred = (model(labels.float())).squeeze()
        test_loss = loss_fun(y_pred,target.float())
        if epoch % 100 == 0:
          print(f"Epoch: {epoch} | Loss: {loss} | Test Loss: {test_loss}")



Epoch: 0 | Loss: 39039283200.0 | Test Loss: 39038971904.0
Epoch: 100 | Loss: 619604864.0 | Test Loss: 618529152.0
Epoch: 200 | Loss: 541143040.0 | Test Loss: 540769216.0
Epoch: 300 | Loss: 522802176.0 | Test Loss: 522719904.0
Epoch: 400 | Loss: 516387872.0 | Test Loss: 516332832.0
Epoch: 500 | Loss: 616103232.0 | Test Loss: 563995072.0
Epoch: 600 | Loss: 511836256.0 | Test Loss: 511795904.0
Epoch: 700 | Loss: 508102496.0 | Test Loss: 508064608.0
Epoch: 800 | Loss: 37210992640.0 | Test Loss: 21247080448.0
Epoch: 900 | Loss: 626211264.0 | Test Loss: 622300352.0


In [122]:
''' evaluate the model '''
model.eval()
with torch.inference_mode():
    y_pred = model(labels.float())
    loss = loss_fun(y_pred,target.float())
    print(f"Loss: {loss}")
''' compare the real value with the predicted one  '''
target[:5],y_pred[:5]

Loss: 6336871936.0


(tensor([208500., 181500., 223500., 140000., 250000.]),
 tensor([[185903.6719],
         [180518.8906],
         [179615.2031],
         [176826.2812],
         [179946.4531]]))

imporving a regressio mode can involve serveral stategies . ehre are some ares for imporvment and adjustments

**use of activation functions**
for regression problems, the last lyer does not have an activaion function  unless youre performing some tranformation on the outpu . howeer the hidden layers usually benefit form non-linear activaion such as ReLu.

**Batch Normalizaion**

batch normalizaion can help speed up training and improve the stability of the model, especialy for deeper networks. it normalizes the inputs
of each layers.

**Dropout**

addiing dropout layers can help prevent overfitting , especailly if your dataset is relatively small or fi yore training for too many epochs

**model complexity**

if your model is too simle , it might not capture the complexity of the data. converselvy it its too complex it might overfit youcan try tweaking the numer of neurons or layesr based on teh datasets complexity .
if you suspect overfitting , reducing the number of layes or using techues like early stoppoong or l2 regulariaion might be eneficial

**learning rate scheduling**

sometimes adjusting the learnign rate throughtout trainign can eard to better convergence. thic can be done through eanrin rate schedulers in pytorch like `ReduceLronplatu`

**advanced optimizaion**

you migh want to experiment with different optimzer like Adam or RAdam, which have adaptive leanring rate that often perform better on regression tasks.

**impoving the loss function**
for regression taks , mean squed error or mean absolute error typically used. you might want to experiment with different loss functonsor add custom loss function if there domain specific knownledge that could help imporve performance.




In [163]:
# imporving the model
class ImprovedRegressionModel(nn.Module):
    def __init__(self):
        super(ImprovedRegressionModel, self).__init__()

        # Define the layers
        self.layers = nn.Sequential(
            nn.Linear(in_features=260, out_features=512),  # Increase the number of neurons
            nn.Linear(in_features=512, out_features=256),

            nn.Linear(in_features=256, out_features=128),


            nn.Linear(in_features=128, out_features=1)  # Output layer
        )

    def forward(self, x):
        return self.layers(x)


In [164]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.L1Loss()  # or nn.L1Loss() for Mean Absolute Error
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)


In [165]:
''' now create a new model and then train it and evaluate it on the new model  '''
improved_model = ImprovedRegressionModel()
for epoch in range(epochs):
    improved_model.train()
    y_pred = (improved_model(labels.float())).squeeze()
    loss = loss_fn(y_pred, target.float())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    improved_model.eval()
    with torch.inference_mode():
        y_pred = improved_model(labels.float())
        test_loss = loss_fn(y_pred, target.float())
        scheduler.step(test_loss)
        if epoch % 100 == 0:
            print(f"Epoch: {epoch} | Loss: {loss} | Test Loss: {test_loss}")


Epoch: 0 | Loss: 180921.15625 | Test Loss: 180921.15625
Epoch: 100 | Loss: 180921.15625 | Test Loss: 180921.15625
Epoch: 200 | Loss: 180921.15625 | Test Loss: 180921.15625
Epoch: 300 | Loss: 180921.15625 | Test Loss: 180921.15625
Epoch: 400 | Loss: 180921.15625 | Test Loss: 180921.15625
Epoch: 500 | Loss: 180921.15625 | Test Loss: 180921.15625
Epoch: 600 | Loss: 180921.15625 | Test Loss: 180921.15625
Epoch: 700 | Loss: 180921.15625 | Test Loss: 180921.15625
Epoch: 800 | Loss: 180921.15625 | Test Loss: 180921.15625
Epoch: 900 | Loss: 180921.15625 | Test Loss: 180921.15625


In [156]:
# evaluat the model
improved_model.eval()
with torch.inference_mode():
    y_pred = (improved_model(labels.float())).squeeze()
    loss = loss_fn(y_pred, target.float())
    print(f"Loss: {loss}")
    print(f"Real Values: {target[:5]}")
    print(f"Predicted Values: {y_pred[:5]}")

Loss: 39039365120.0
Real Values: tensor([208500., 181500., 223500., 140000., 250000.])
Predicted Values: tensor([-0.6518, -0.5472, -0.2802, -0.5107, -0.4834])


In [166]:
labels, target

(tensor([[ 0.0734, -0.2294, -0.2071,  ...,  0.0000,  1.0000,  0.0000],
         [-0.8726,  0.4519, -0.0919,  ...,  0.0000,  1.0000,  0.0000],
         [ 0.0734, -0.0931,  0.0735,  ...,  0.0000,  1.0000,  0.0000],
         ...,
         [ 0.3099, -0.1840, -0.1478,  ...,  0.0000,  1.0000,  0.0000],
         [-0.8726, -0.0931, -0.0802,  ...,  0.0000,  1.0000,  0.0000],
         [-0.8726,  0.2248, -0.0581,  ...,  0.0000,  1.0000,  0.0000]]),
 tensor([208500., 181500., 223500.,  ..., 266500., 142125., 147500.]))

In [167]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Step 1: Create a simple neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(SimpleNN, self).__init__()
        # Define layers (you can adjust hidden_dim to tune the model)
        self.fc1 = nn.Linear(input_dim, hidden_dim)  # Input to hidden layer
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)  # Hidden to hidden layer
        self.fc3 = nn.Linear(hidden_dim, 1)  # Hidden to output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Apply ReLU activation
        x = torch.relu(self.fc2(x))  # Apply ReLU activation
        x = self.fc3(x)  # Output layer (no activation for regression)
        return x

# Step 2: Prepare the data
# Assuming 'inputs' is the first tensor and 'targets' is the second tensor

# Create a Dataset and DataLoader for batching
dataset = TensorDataset(labels, target)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# Step 3: Instantiate the model, loss function, and optimizer
input_dim = labels.shape[1]  # Number of features
model = SimpleNN(input_dim)

# Use Mean Squared Error loss for regression
criterion = nn.MSELoss()

# Optimizer (using Adam)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Step 4: Define the training loop
def train(model, dataloader, criterion, optimizer, num_epochs=20):
    model.train()  # Set the model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for inputs_batch, targets_batch in dataloader:
            # Zero the gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs_batch)

            # Compute the loss
            loss = criterion(outputs.squeeze(), targets_batch)

            # Backward pass (gradient calculation)
            loss.backward()

            # Update the weights
            optimizer.step()

            # Accumulate loss
            running_loss += loss.item()

        # Average loss for the epoch
        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Step 5: Train the model
num_epochs = 20
train(model, dataloader, criterion, optimizer, num_epochs)

# Step 6: Evaluation (Optional)
# After training, you can evaluate the model on some validation data (here using the training data itself)
def evaluate(model, inputs, targets):
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():  # No need to compute gradients during evaluation
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), targets)
    return loss.item()

train_loss = evaluate(model, labels, target)
print(f"Final training loss: {train_loss:.4f}")


Epoch 1/20, Loss: 39038116285.2174
Epoch 2/20, Loss: 39039378387.4783
Epoch 3/20, Loss: 39058447315.4783
Epoch 4/20, Loss: 38807082295.6522
Epoch 5/20, Loss: 38550606180.1739
Epoch 6/20, Loss: 38057604140.5217
Epoch 7/20, Loss: 37320272673.3913
Epoch 8/20, Loss: 36143394726.9565
Epoch 9/20, Loss: 34536159855.3043
Epoch 10/20, Loss: 32512513068.5217
Epoch 11/20, Loss: 30164418470.9565
Epoch 12/20, Loss: 27294188410.4348
Epoch 13/20, Loss: 24231993878.2609
Epoch 14/20, Loss: 20931671396.1739
Epoch 15/20, Loss: 17607668736.0000
Epoch 16/20, Loss: 14325658468.1739
Epoch 17/20, Loss: 11347807387.8261
Epoch 18/20, Loss: 8735504873.7391
Epoch 19/20, Loss: 6569794726.9565
Epoch 20/20, Loss: 4911275898.4348
Final training loss: 4260528384.0000


In [ ]:
# you need to do more reserch your model is not performing bro